# [튜토리얼 6] 모델 저장과 복원

이번 튜토리얼에서는 모델을 저장하고, 저장한 모델을 복원하는 연습을 해보겠습니다. 훈련하는 도중이나 훈련이 끝난 후에 모델을 저장할 수 있는데, 모델을 저장하고, 복원하게 되면 중지된 지점부터 다시 훈련할 수 있기 때문에 한 번에 오랫동안 훈련하지 않아도 된다는 장점이 있습니다. 또한, 모델을 저장하면 다른 사람에게 공유할 수 있고 작업을 재현할 수도 있습니다. 따라서 연구한 모델과 기법을 공개할 때 많은 머신 러닝 기술자들이 다음과 같은 사항을 제공합니다:

* 모델을 만드는 코드
* 모델의 훈련된 가중치 또는 파라미터

이런 데이터를 공유하면 다른 사람들이 모델의 작동 방식을 이해하고 새로운 데이터로 모델을 실험하는데 도움이 됩니다.

**주의**: 신뢰할 수 없는 코드는 조심하세요. 텐서플로우 모델은 프로그램 코드입니다.

**< 저장 방식 >**

사용하는 API에 따라서 여러가지 방법으로 텐서플로 모델을 저장할 수 있습니다. 

In [ ]:
#필요한 라이브러리 임포트

import warnings
warnings.simplefilter('ignore')

import os
import tensorflow as tf
from tensorflow import keras

tf.__version__

# 목차
1. 예제 데이터셋 받기
2. 모델 정의
3. 훈련하는 동안 체크포인트 저장하기
4. 체크포인트 파일
5. 수동으로 가중치 저장하기
6. 모델 전체를 저장하기
    - 6.1. HDF5 파일로 저장하기
    - 6.2. saved_model을 사용하기

## 1. 예제 데이터셋 받기

손글씨 데이터셋인 [MNIST 데이터셋](http://yann.lecun.com/exdb/mnist/)으로 모델을 훈련하여 가중치를 저장하는 예제를 만들어 보겠습니다. MNIST 데이터셋은 28x28 픽셀의 흑백 이미지 데이터셋으로 60,000개의 훈련 이미지와 10,000개의 테스트 이미지를 포함합니다.
이번 예제에서는 모델 실행 속도를 빠르게 하기 위해 샘플에서 훈련 이미지와 테스트 이미지 각각 처음 1,000개만 사용하겠습니다:

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

## 2. 모델 정의

가중치를 저장하고 불러오는 예제를 진행하기 위해 간단한 모델을 만들어 봅시다.

In [ ]:
# 간단한 Sequential 모델을 반환합니다
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    return model


# 모델 객체를 만듭니다
model = create_model()
model.summary()

## 3. 훈련하는 동안 체크포인트 저장하기

많이 사용하는 방법은 **훈련 중간**과 **훈련 마지막**에 체크포인트(checkpoint)를 자동으로 저장하도록 하는 것 입니다. 체크 포인트를 자동으로 저장하게 되면, 다시 훈련하지 않고 모델을 재사용하거나 훈련 과정이 중지된 경우 이어서 훈련을 진행할 수 있습니다.

`tf.keras.callbacks.ModelCheckpoint`은 이런 작업을 수행하는 콜백(callback)입니다. 이 콜백은 체크포인트 작업을 조정할 수 있도록 여러가지 매개변수를 제공합니다.

### 3.1 체크포인트 콜백 사용하기

`ModelCheckpoint` 콜백을 전달하여 모델을 훈련해 봅시다:

In [ ]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 체크포인트 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10,
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # 훈련 단계에 콜백을 전달합니다

이 코드는 텐서플로 체크포인트 파일을 만들고 에포크가 종료될 때마다 업데이트합니다:

In [ ]:
!ls {checkpoint_dir}

훈련하지 않은 새로운 모델을 만들어 보겠습니다. 가중치만 복원할 땐 원본 모델과 동일한 구조로 모델을 만들어야 합니다. 여기서는 동일한 구조로 모델을 만들었기 때문에 다른 **객체**이지만 가중치를 공유할 수 있습니다.

훈련하지 않은 새 모델을 만들고 테스트 세트에서 평가해 봅시다. 훈련하지 않은 모델의 성능은 레이블을 무작위로 선택하는 정도의 수준입니다(~10% 정확도):

In [ ]:
model = create_model()

loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

체크포인트에서 가중치를 로드하고 다시 평가해 봅시다:

In [ ]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

### 3.2 체크포인트 콜백 매개변수

이 콜백 함수는 몇 가지 매개변수를 제공합니다. 이러한 매개변수들을 통해 체크포인트 이름을 고유하게 만들거나 체크포인트 주기를 조정할 수 있습니다.

새로운 모델을 훈련하고 5번의 에포크마다 고유한 이름으로 체크포인트를 저장해 보겠습니다:

In [ ]:
# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # 다섯 번째 에포크마다 가중치를 저장합니다
    period=5)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)

만들어진 체크포인트를 확인해 보고 마지막 체크포인트를 선택해 보겠습니다:

In [ ]:
! ls {checkpoint_dir}

In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

텐서플로우는 기본적으로 최근 **5개의 체크포인트**만 저장합니다.

모델을 초기화하고 최근 체크포인트를 로드하여 테스트해 보겠습니다:

In [ ]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

## 4. 체크포인트 파일

위 코드는 가중치를 일련의 체크포인트 포맷의 파일에 저장합니다. 이 파일에 포함되는 것은 훈련된 이진 포맷의 가중치입니다. 체크포인트가 담고 있는 것은 다음과 같습니다:

* 모델의 가중치를 포함하는 하나 이상의 샤드(shard)
* 가중치가 어느 샤드에 저장되어 있는지를 나타내는 인덱스 파일

단일 머신에서 모델을 훈련한다면 `.data-00000-of-00001` 확장자를 가진 샤드 하나만 만들어 집니다.

## 5. 수동으로 가중치 저장하기

앞에서 가중치를 모델에 로드하는 방법을 보았습니다.

물론 수동으로 가중치를 저장하는 것도 가능합니다. `Model.save_weights` 메서드를 사용합니다.

In [ ]:
# 가중치를 저장합니다
model.save_weights('./checkpoints/my_checkpoint')

# 가중치를 복원합니다
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

## 6. 모델 전체를 저장하기

혹은 전체 모델을 파일 하나에 저장할 수도 있습니다. 여기에는 가중치, 모델 구성 심지어 옵티마이저에 지정한 설정까지 포함됩니다. 모델의 체크포인트를 저장하므로 원본 코드를 사용하지 않고도 나중에 정확히 동일한 상태에서 훈련을 다시 시작할 수 있습니다.

전체 모델을 저장하는 기능은 매우 유용합니다. 전체 모델 저장 기능을 사용하면 TensorFlow.js로 모델을 로드한 다음 웹 브라우저에서 모델을 훈련하고 실행할 수 있습니다(HDF5, Saved Model). 또는 모바일 장치에 맞도록 변환한 다음 TensorFlow Lite를 사용하여 실행할 수도 있습니다(HDF5, Saved Model).

### 6.1 HDF5 파일로 저장하기

케라스는 HDF5 표준을 따르는 기본 저장 포맷을 제공합니다. HDF5 파일로 저장된 모델을 하나의 이진 파일(binary blob)처럼 다룰 수 있습니다.

In [ ]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# 전체 모델을 HDF5 파일로 저장합니다
model.save('my_model.h5')

이제 이 파일로부터 모델을 다시 만들어 봅시다:

In [ ]:
# 가중치와 옵티마이저를 포함하여 정확히 동일한 모델을 다시 생성합니다
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

정확도를 확인해 보겠습니다:

In [ ]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

이 기법은 다음의 모든 것을 저장합니다:

* 가중치 값
* 모델 설정(구조)
* 옵티마이저 설정

케라스는 모델 구조를 확인하고 저장합니다. 현재는 텐서플로 옵티마이저(`tf.train`)를 저장할 수 없습니다. 이런 경우에는 모델을 로드한 후에 다시 컴파일해야 합니다. 옵티마이저의 상태는 유지되지 않습니다.

### 6.2 `saved_model`을 사용하기

새로운 모델을 만들어 보겠습니다:

In [ ]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

`saved_model`을 만들어 타임스탬프를 이름으로 가진 디렉토리에 저장합니다:

In [ ]:
import time
saved_model_path = "./saved_models/{}".format(int(time.time()))

tf.keras.models.save_model(model, saved_model_path)
saved_model_path

저장된 모델의 목록을 확인합니다:

In [ ]:
!ls saved_models/

저장된 모델로부터 새로운 케라스 모델을 로드합니다.

In [ ]:
new_model = tf.keras.models.load_model(saved_model_path)
new_model.summary()

복원된 모델을 실행합니다.

In [ ]:
model.predict(test_images).shape

In [ ]:
# 이 모델을 평가하려면 그전에 컴파일해야 합니다.
# 단지 저장된 모델의 배포라면 이 단계가 필요하지 않습니다.

new_model.compile(optimizer=model.optimizer, # 복원된 옵티마이저를 사용합니다.
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# 복원된 모델을 평가합니다
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

# Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.